In [1]:
# importing and installing 
import pandas as pd



import openai
import pandas as pd
import time




In [2]:
# loading dataset



# Directory path where the thread files are stored
path = "../newdataset/updated_data.csv"
data = pd.read_csv(path)

# Check the dataset
print(f"Dataset loaded successfully with {len(data)} rows.")
print(data.head())


Dataset loaded successfully with 53983 rows.
  comment_id  link_id   subreddit_x   parent_id         redditor_id_x  \
0    k3d3suw  16yqq5w  changemyview  t3_16yqq5w                10qxmu   
1    k3anc3z  16yqq5w  changemyview  t3_16yqq5w               deleted   
2    k3aek0j  16yqq5w  changemyview  t3_16yqq5w                 5kww6   
3    k3a4ekd  16yqq5w  changemyview  t3_16yqq5w                 4bnx7   
4    k39xcxh  16yqq5w  changemyview  t3_16yqq5w  suspended:barbodelli   

                created_at_x  \
0  2023-10-04T02:32:48+00:00   
1  2023-10-03T17:34:08+00:00   
2  2023-10-03T16:40:00+00:00   
3  2023-10-03T15:32:05+00:00   
4  2023-10-03T14:38:43+00:00   

                                                body  \
0  This post has been locked due to a large numbe...   
1  I'll share an anecdote. I'm well read enough t...   
2  Please forgive not having links. I wrote this ...   
3  You’re not quite thinking about this the right...   
4  [https://cei.org/blog/wrong-again-50-yea

In [5]:
# Select the first 2000 rows but keep all columns
sample_data = data.head(2000)

# Check the sample
sample_data[['cleaned_text','cleaned_body']].head(5)

,cleaned_text,cleaned_body
0,when you ask the average conservative why they...,this post has been locked due to a large numbe...
1,when you ask the average conservative why they...,i'll share an anecdote. i'm well read enough t...
2,when you ask the average conservative why they...,please forgive not having links. i wrote this ...
3,when you ask the average conservative why they...,youre not quite thinking about this the right ...
4,when you ask the average conservative why they...,how do you tell the difference between actual ...


In [7]:
import openai

In [ ]:

import time

# Set up OpenAI API key
openai.api_key = ""


# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the semantic relationship between the following post and comment.
        Post: {post}
        Comment: {comment}
        
        Does the comment:
        +1: Agree with the post,
        -1: Disagree with the post,
         0: Neutral (neither agree nor disagree)?
        
        Respond with one number only: +1, -1, or 0.
        """
        
        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract the response content
        label = response['choices'][0]['message']['content'].strip()
        return int(label)  # Convert to integer

    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None if there's an error

# Process the dataset (use a sample for testing, e.g., first 2000 rows)
sample_data = data.head(2000).copy()

# Add a new column for GPT labels
sample_data['label'] = None

# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label
    label = get_gpt_label(post, comment)
    sample_data.at[i, 'label'] = label
    
    # Optional: Print progress
    if i % 10 == 0:
        print(f"Processed {i} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(1.5)

# Save the labeled dataset
output_path = "labeled_sample_data.csv"
sample_data.to_csv(output_path, index=False)
print(f"Labeled dataset saved to {output_path}")


In [ ]:

import time

# Set up OpenAI API key
openai.api_key = ""


# Function to get GPT label for a single post and comment
def get_gpt_label(post, comment):
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the semantic relationship between the following post and comment.
        Post: {post}
        Comment: {comment}
        
        Does the comment:
        +1: Agree with the post,
        0: Disagree with the post,
        
        
        Respond with one number only: +1 or 0.
        """
        
        # Call OpenAI GPT API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract the response content
        label = response['choices'][0]['message']['content'].strip()
        return int(label)  # Convert to integer

    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None if there's an error

# Process the dataset (use a sample for testing, e.g., first 2000 rows)
sample_data = data.head(2000).copy()

# Add a new column for GPT labels
sample_data['label_binary'] = None

# Iterate through each post and comment pair
for i, row in sample_data.iterrows():
    post = row['cleaned_text']
    comment = row['cleaned_body']
    
    # Get GPT label
    label = get_gpt_label(post, comment)
    sample_data.at[i, 'label_binary'] = label
    
    # Optional: Print progress
    if i % 10 == 0:
        print(f"Processed {i} rows...")

    # Avoid hitting API rate limits (adjust delay as needed)
    time.sleep(1.5)

# Save the labeled dataset
output_path = "labeled_sample_data2.csv"
sample_data.to_csv(output_path, index=False)
print(f"Labeled dataset saved to {output_path}")


In [9]:
# Define truncation function
def truncate_text(text, max_length=500):
    if isinstance(text, str):
        words = text.split()
        return " ".join(words[:max_length]) if len(words) > max_length else text
    return text

# Apply truncation
data['cleaned_text'] = data['cleaned_text'].apply(lambda x: truncate_text(x, 500))
data['cleaned_body'] = data['cleaned_body'].apply(lambda x: truncate_text(x, 500))

# Check a sample after truncation
print(data[['cleaned_text', 'cleaned_body']].head())


                                        cleaned_text  \
0  when you ask the average conservative why they...   
1  when you ask the average conservative why they...   
2  when you ask the average conservative why they...   
3  when you ask the average conservative why they...   
4  when you ask the average conservative why they...   

                                        cleaned_body  
0  this post has been locked due to a large numbe...  
1  i'll share an anecdote. i'm well read enough t...  
2  please forgive not having links. i wrote this ...  
3  youre not quite thinking about this the right ...  
4  how do you tell the difference between actual ...  


In [11]:
# Group comments under the same post
grouped_data = data.groupby('cleaned_text')['cleaned_body'].apply(list).reset_index()

# Preview grouped data
print(grouped_data.head())


                                        cleaned_text  \
0  ( note: this is where i got this opinion from....   
1  (disclaimer: i'm not canadian; i'm american. i...   
2  (edited to note that im concerned with the rig...   
3  0.7 is way too high. we're far from being able...   
4  1 the population collapse would be bad in the ...   

                                        cleaned_body  
0  [if trump had the same level of power trudeau ...  
1  [russia especially benefits from global warmin...  
2  [nan, so if you are wondering whether astrolog...  
3  [i think the kardashev scale is a bitsilly. we...  
4  [in 1989, year of ussr collapsed and romania g...  


In [13]:
def create_gpt_prompt(post, comments):
    # Example template
    examples = """
Examples:
Post: "Climate change is urgent and requires attention."
Comment: "Absolutely, we need better policies." -> +1 (Agree)
Comment: "I don't think it's that big of a deal." -> -1 (Disagree)
Comment: "It's hard to say." -> 0 (Neutral)
    """
    
    # Format comments for analysis
    formatted_comments = "\n".join([f"Comment: {c}" for c in comments])
    
    # Create the full prompt
    prompt = f"{examples}\n\nNow analyze the following:\nPost: {post}\n{formatted_comments}\nLabel:"
    return prompt


In [ ]:
import openai
import time

# Set up OpenAI API key
openai.api_key = ""

# Function to label a post and its comments
def label_comments(post, comments):
    try:
        prompt = create_gpt_prompt(post, comments)
        
        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract labels
        labels = response['choices'][0]['message']['content'].strip().split("\n")
        return labels
    except Exception as e:
        print(f"Error: {e}")
        return [None] * len(comments)

# Test on a single group (first post with its comments)
post = grouped_data['cleaned_text'][0]
comments = grouped_data['cleaned_body'][0]
labels = label_comments(post, comments)

# Display labels
print(labels)


['-1 (Disagree)']


In [16]:
# Add labels to the dataset
grouped_data['labels'] = grouped_data.apply(
    lambda row: label_comments(row['cleaned_text'], row['cleaned_body']), axis=1
)




Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Error

In [ ]:


def create_gpt_prompt_binary(post, comments):
    # Example template
    examples = """
Examples:
Post: "Climate change is urgent and requires attention."
Comment: "Absolutely, we need better policies." -> 1 (Agree)
Comment: "I don't think it's that big of a deal." -> 0 (Disagree/Neutral)
    """
    
    # Format comments for analysis
    formatted_comments = "\n".join([f"Comment: {c}" for c in comments])
    
    # Create the full prompt
    prompt = f"{examples}\n\nNow analyze the following:\nPost: {post}\n{formatted_comments}\nLabel:"
    return prompt


In [ ]:
def label_comments_binary(post, comments):
    try:
        prompt = create_gpt_prompt_binary(post, comments)
        
        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract binary labels
        labels = response['choices'][0]['message']['content'].strip().split("\n")
        
        # Map textual response to binary classification
        binary_labels = [1 if label.strip() == "1" else 0 for label in labels]
        return binary_labels
    except Exception as e:
        print(f"Error: {e}")
        return [None] * len(comments)


In [ ]:
# Add binary labels to the dataset
grouped_data['binary_labels'] = grouped_data.apply(
    lambda row: label_comments_binary(row['cleaned_text'], row['cleaned_body']), axis=1
)

# Save the binary labeled dataset
grouped_data.to_csv("binary_labeled_dataset.csv", index=False)
print("Binary labeled dataset saved.")
